<a href="https://colab.research.google.com/github/kyochanpy/Google_Smartphone_Decimeter_Challenge/blob/main/defs/01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install optuna > /dev/null
!pip install pyproj > /dev/null
!pip install simdkalman > /dev/null
    
import os
from glob import glob
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns

import optuna
import plotly
import plotly.express as px
import pyproj
from pathlib import Path
from pyproj import Proj, transform
from tqdm.notebook import tqdm
import simdkalman

In [4]:
path = Path("/content/drive/MyDrive/GSDC")
test_base = pd.read_csv(path / "baseline_locations_test.csv")
sub = pd.read_csv(path / "sample_submission.csv")

truths = (path / "train").rglob("ground_truth.csv")

df_list = []
cols = ["collectionName", "phoneName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]

for t in tqdm(truths, total=73):
    df_phone = pd.read_csv(t, usecols=cols)
    df_list.append(df_phone)
df_truth = pd.concat(df_list, ignore_index=True)

train_base = pd.read_csv(path / "baseline_locations_train.csv", usecols=cols)
all_df = df_truth.merge(train_base, how="inner", on=cols[:3], suffixes=("_truth", '_train_base'))

# Check Score

In [5]:
def check_score(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.t_latDeg, r.t_lngDeg
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()
    print(f'error meter: {meter_score}')

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    print(f'score: {score}')
    
    return output_df

# calc_haversine

In [6]:
def calc_haversine(lat1, lon1, lat2, lon2):
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    d = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(d**0.5)
    return dist

# Mean Prediction

In [7]:
def mean_prediction(input_df):
    def make_lerp_data(input_df):
        org_colus = input_df.columns

        time_list = input_df[["collectionName", "millisSinceGpsEpoch"]].drop_duplicates()
        phone_list = input_df[["collectionName", "phoneName"]].drop_duplicates()
        tmp = time_list.merge(phone_list, on="collectionName", how="outer")

        output_df = tmp.merge(input_df, on=["collectionName", "millisSinceGpsEpoch", "phoneName"], how="left")
        output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]
        output_df = output_df.sort_values(["phone", "millisSinceGpsEpoch"])

        output_df["latDeg_pre"] = output_df["latDeg"].shift(1)
        output_df["latDeg_pro"] = output_df["latDeg"].shift(-1)
        output_df["lngDeg_pre"] = output_df["lngDeg"].shift(1)
        output_df["lngDeg_pro"] = output_df["lngDeg"].shift(-1)
        output_df["phone_pre"] = output_df["phone"].shift(1)
        output_df["phone_pro"] = output_df["phone"].shift(-1)
        output_df["millisSinceGpsEpoch_pre"] = output_df["millisSinceGpsEpoch"].shift(1)
        output_df["millisSinceGpsEpoch_pro"] = output_df["millisSinceGpsEpoch"].shift(-1)

        output_df = output_df[(output_df["latDeg"].isnull())&(output_df["phone"] == output_df["phone_pre"])&
                            (output_df["phone"] == output_df["phone_pro"])].copy()

        output_df["latDeg"] = output_df["latDeg_pre"] + ((output_df["latDeg_pro"] - output_df["latDeg_pre"]) * 
                                                        ((output_df["millisSinceGpsEpoch"] - output_df["millisSinceGpsEpoch_pre"]) /
                                                        (output_df["millisSinceGpsEpoch_pro"] - output_df["millisSinceGpsEpoch_pre"])))
        output_df["lngDeg"] = output_df["lngDeg_pre"] + ((output_df["lngDeg_pro"] - output_df["lngDeg_pre"]) * 
                                                        ((output_df["millisSinceGpsEpoch"] - output_df["millisSinceGpsEpoch_pre"]) /
                                                        (output_df["millisSinceGpsEpoch_pro"] - output_df["millisSinceGpsEpoch_pre"])))
        
        output_df = output_df[~output_df['latDeg'].isnull()]

        return output_df[org_colus]

    
    def calc_mean_pred(input_df, lerp_df):
        input_df["phone"] = input_df["collectionName"] + "_" + input_df["phoneName"]
        add_lerp = pd.concat([input_df, lerp_df])
        mean_pred_result = add_lerp.groupby(["collectionName", "millisSinceGpsEpoch"])[["latDeg", "lngDeg"]].mean().reset_index()
        output_df = input_df[["collectionName", "phoneName", "millisSinceGpsEpoch"]].copy()
        output_df = output_df.merge(mean_pred_result[["collectionName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]],
                                        on=["collectionName", "millisSinceGpsEpoch"], how="left")
        output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]
        return output_df


    test_lerp = make_lerp_data(input_df)
    test_mean_pred  = calc_mean_pred(input_df, test_lerp)

    sub["latDeg"] = test_mean_pred["latDeg"]
    sub["lngDeg"] = test_mean_pred["lngDeg"]

    output_df = sub.copy()
    output_df["phoneName"] = input_df["phoneName"]
    output_df["collectionName"] = input_df["collectionName"]
    output_df["millisSinceGpsEpoch"] = input_df["millisSinceGpsEpoch"]

    return output_df

# Outlier 

In [8]:
def outlier_test_per_95(input_df):
    output_df = input_df

    def calc_haversine(lat1, lon1, lat2, lon2):
        RADIUS = 6_367_000
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        d = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        dist = 2 * RADIUS * np.arcsin(d**0.5)
        return dist

    output_df["dist_pre"] = 0
    output_df["dist_pro"] = 0

    output_df['latDeg_pre'] = output_df['latDeg'].shift(periods=1,fill_value=0)
    output_df['lngDeg_pre'] = output_df['lngDeg'].shift(periods=1,fill_value=0)
    output_df['latDeg_pro'] = output_df['latDeg'].shift(periods=-1,fill_value=0)
    output_df['lngDeg_pro'] = output_df['lngDeg'].shift(periods=-1,fill_value=0)
    output_df['dist_pre'] = calc_haversine(output_df.latDeg_pre, output_df.lngDeg_pre, output_df.latDeg, output_df.lngDeg)
    output_df['dist_pro'] = calc_haversine(output_df.latDeg, output_df.lngDeg, output_df.latDeg_pro, output_df.lngDeg_pro)

    list_phone = output_df['phone'].unique()
    for phone in list_phone:
        ind_s = output_df[test_base['phone'] == phone].index[0]
        ind_e = output_df[test_base['phone'] == phone].index[-1]
        output_df.loc[ind_s,'dist_pre'] = 0
        output_df.loc[ind_e,'dist_pro'] = 0

    pro_95 = output_df['dist_pro'].mean() + (output_df['dist_pro'].std() * 2)
    pre_95 = output_df['dist_pre'].mean() + (output_df['dist_pre'].std() * 2)
    ind = output_df[(output_df['dist_pro'] > pro_95)&(output_df['dist_pre'] > pre_95)][['dist_pre','dist_pro']].index

    for i in ind:
        output_df.loc[i,'latDeg'] = (output_df.loc[i-1,'latDeg'] + output_df.loc[i+1,'latDeg'])/2
        output_df.loc[i,'lngDeg'] = (output_df.loc[i-1,'lngDeg'] + output_df.loc[i+1,'lngDeg'])/2

    output_df["phoneName"] = input_df["phoneName"]
    output_df["collectionName"] = input_df["collectionName"]
    output_df["millisSinceGpsEpoch"] = input_df["millisSinceGpsEpoch"]

    return output_df

# Position Shift

In [9]:
def position_shift(input_df):
    sub_cols = sub.columns

    train_p_s = pd.read_csv(path / "baseline_locations_train.csv")
    train_b = train_p_s[sub_cols]
    test_b = test_base[sub_cols]

    msge = "millisSinceGpsEpoch"

    datadir = Path('/content/drive/MyDrive/GSDC')
    testdir = datadir / 'test'
    traindir = datadir / 'train'

    g_t = pd.DataFrame()
    for d in os.listdir(traindir):
        for p in os.listdir(traindir/d):
            g_t = g_t.append(pd.read_csv(traindir/d/p/'ground_truth.csv'))

    g_t["phone"] = g_t["collectionName"] + "_" + g_t["phoneName"]
    g_t_sub_cols = g_t[sub_cols]


    def compute_dist(fname_1, fname_2=g_t):
        oof = fname_1
        g_t = fname_2
        df = oof.merge(g_t, on=["phone", "millisSinceGpsEpoch"])
        dist_oof = calc_haversine(df.latDeg_x, df.lngDeg_x, df.latDeg_y, df.lngDeg_y)
        scores = pd.DataFrame({"phone":df.phone, "dist":dist_oof})
        scores_grp = scores.groupby("phone")
        d_50 = scores_grp.quantile(.50).reset_index()
        d_50.columns = ["phone", "q_50"]
        d_95 = scores_grp.quantile(.95).reset_index()
        d_95.columns = ["phone", "q_95"]
        return (scores_grp.quantile(.50).mean() + scores_grp.quantile(.95).mean()) / 2, d_50.merge(d_95)


    def WGS84_to_ECEF(lat, lon, alt):
        rad_lat = lat * (np.pi / 180.0)
        rad_lon = lon * (np.pi / 180.0)
        a = 6378137.0
        finv = 298.257223563
        f = 1 / finv   
        e2 = 1 - (1 - f) * (1 - f)    
        N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
        x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lon)
        y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lon)
        z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
        return x, y, z    


    transformer = pyproj.Transformer.from_crs(
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)


    def ECEF_to_WGS84(x, y, z):
        lon, lat, alt = transformer.transform(x, y, z, radians=False)
        return lon, lat, alt


    def position_shift_(input_df, a):
        output_df = input_df.copy()
        output_df["heightAboveWgs84EllipsoidM"] = 63.5
        output_df["x"], output_df["y"], output_df["z"] = zip(*output_df.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, 
                                                                                                    x.heightAboveWgs84EllipsoidM), axis=1))
        output_df.sort_values(["phone", msge], inplace=True)
        for fi in ["x", "y", "z"]:
            output_df[[fi + "_p"]] = output_df[fi].shift().where(output_df["phone"].eq(output_df["phone"].shift()))
            output_df[[fi + "_diff"]] = output_df[fi] - output_df[fi + "_p"]

        output_df[["dist"]] = np.sqrt(output_df["x_diff"]**2 + output_df["y_diff"]**2 + output_df["z_diff"]**2)
        for fi in ["x", "y", "z"]:
            output_df[[fi + "_new"]] = output_df[fi + "_p"] + output_df[fi + "_diff"] * (1 - a/output_df["dist"])
        lng, lat, alt = ECEF_to_WGS84(output_df["x_new"].values, output_df["y_new"].values, output_df["z_new"].values)

        lat[np.isnan(lat)] = output_df.loc[np.isnan(lat), "latDeg"]
        lng[np.isnan(lng)] = output_df.loc[np.isnan(lng), "lngDeg"]
        output_df["latDeg"] = lat
        output_df["lngDeg"] = lng

        output_df.sort_values(["phone", msge], inplace=True)

        return output_df[sub_cols]


    def objective(trial):
        a = trial.suggest_uniform("a", -1, 1)
        score, scores = compute_dist(position_shift_(train_b, a), g_t)
        return score


    study = optuna.create_study()
    study.optimize(objective, n_trials=30)

    output_df = position_shift_(test_base, a=study.best_params["a"])

    output_df["phoneName"] = input_df["phoneName"]
    output_df["collectionName"] = input_df["collectionName"]
    output_df["millisSinceGpsEpoch"] = input_df["millisSinceGpsEpoch"]

    return output_df

# Remove Device

In [10]:
def remove_device(input_df):

    def get_groundtruth(path: Path) -> pd.DataFrame:
        output_df = pd.DataFrame()
        
        for path in glob(str(path / 'train/*/*/ground_truth.csv')):
            _df = pd.read_csv(path)
            output_df = pd.concat([output_df, _df])
        output_df = output_df.reset_index(drop=True)
        
        _columns = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']
        output_df[['t_'+col for col in _columns]] = output_df[_columns]
        output_df = output_df.drop(columns=_columns, axis=1)
        return output_df

    train_base = pd.read_csv(path / "baseline_locations_train.csv", usecols=cols)
    train_base = train_base.merge(
    get_groundtruth(path),
    on=['collectionName', 'phoneName', 'millisSinceGpsEpoch']
    )

    remove_list = []
    for name, df in train_base.groupby('collectionName'):
        phones_list = df["phoneName"].unique().tolist()
        meter_list = []

        if (len(phones_list) == 1):
                remove_list.append('dont_remove')
                continue

        for phone in phones_list:
            phone_df = df[df["phoneName"] == phone]
            phone_df['meter'] = phone_df.apply(lambda r: calc_haversine(r.latDeg, r.lngDeg, r.t_latDeg, r.t_lngDeg),axis=1)
            meter_score = phone_df['meter'].mean()
            meter_list.append(meter_score)
        remove_meter = sorted(meter_list)[-1]
        remove_idx = meter_list.index(remove_meter)
        remove_list.append(phones_list[remove_idx])


    def get_removedevice(input_df):
        input_df['index'] = input_df.index
        input_df = input_df.sort_values('millisSinceGpsEpoch')
        input_df.index = input_df['millisSinceGpsEpoch'].values
        sub_list = list(input_df["collectionName"].unique())

        output_df = pd.DataFrame() 
        for i, subdf in input_df.groupby('collectionName'):

            phones = subdf['phoneName'].unique()
            sub_idx = sub_list.index(i)
            device = remove_list[sub_idx-1]

            if (len(phones) == 1) or (not device in phones):
                output_df = pd.concat([output_df, subdf])
                continue

            origin_df = subdf.copy()
                
            _index = subdf['phoneName']==device
            subdf.loc[_index, 'latDeg'] = np.nan
            subdf.loc[_index, 'lngDeg'] = np.nan
            subdf = subdf.interpolate(method='index', limit_area='inside')

            _index = subdf['latDeg'].isnull()
            subdf.loc[_index, 'latDeg'] = origin_df.loc[_index, 'latDeg'].values
            subdf.loc[_index, 'lngDeg'] = origin_df.loc[_index, 'lngDeg'].values

            output_df = pd.concat([output_df, subdf])

        output_df.index = output_df['index'].values
        output_df = output_df.sort_index()

        del output_df['index']
            
        return output_df


    output_df = get_removedevice(input_df)


    output_df["phoneName"] = input_df["phoneName"]
    output_df["collectionName"] = input_df["collectionName"]
    output_df["millisSinceGpsEpoch"] = input_df["millisSinceGpsEpoch"]
    
    return output_df

# Kalman Filter

In [11]:
def kalman_filter(input_df):
    
    T = 1.0
    state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                                [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
    process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
    observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
    observation_noise = np.diag([1e-4, 1e-4]) + np.ones((2, 2)) * 1e-9

    kf = simdkalman.KalmanFilter(
            state_transition = state_transition,
            process_noise = process_noise,
            observation_model = observation_model,
            observation_noise = observation_noise)
    
    def apply_kf_smoothing(df, kf_=kf):
        unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
        for collection, phone in tqdm(unique_paths):
            cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
            data = df[cond][['latDeg', 'lngDeg']].to_numpy()
            data = data.reshape(1, len(data), 2)
            smoothed = kf_.smooth(data)
            df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
            df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
        return df

    kf_smoothed_baseline = apply_kf_smoothing(input_df)
    output_df = sub.assign(latDeg = kf_smoothed_baseline.latDeg, lngDeg = kf_smoothed_baseline.lngDeg)
    output_df["phoneName"] = input_df["phoneName"]
    output_df["collectionName"] = input_df["collectionName"]
    output_df["millisSinceGpsEpoch"] = input_df["millisSinceGpsEpoch"]
    
    return output_df

# SUB

In [ ]:
submission_1 = outlier_test_per_95(train))
submission_2 = kalman_filter(test_base)
submission_3 = mean_prediction(test_base)
submission_4 = remove_device(test_base)
submission_5 = position_shift(test_base)

In [12]:
submission_4 = remove_device(test_base)
submission_5 = position_shift(test_base)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

[I 2021-06-29 07:11:29,691] A new study created in memory with name: no-name-e720c3de-389b-459d-b950-23bc52337956
[I 2021-06-29 07:11:36,248] Trial 0 finished with value: 5.208480915832899 and parameters: {'a': 0.3383851974501615}. Best is trial 0 with value: 5.208480915832899.
[I 2021-06-29 07:11:42,662] Trial 1 finished with value: 5.615642342866335 and parameters: {'a': -0.7594864489444135}. Best is trial 0 with value: 5.208480915832899.
[I 2021-06-29 07:11:49,299] Trial 2 finished with value: 5.188384777486013 and parameters: {'a': 0.6851136337219226}. Best is trial 2 with value: 5.188384777486013.
[I 2021-06-29 07:11:55,654] Trial 3

In [48]:
submission_1[sub.columns].to_csv('submission_fake_06.csv', index=False)

In [49]:
submission_2[sub.columns].to_csv('submission_fake_07.csv', index=False)

In [50]:
submission_3[sub.columns].to_csv('submission_fake_08.csv', index=False)

In [13]:
submission_4[sub.columns].to_csv('submission_fake_09.csv', index=False)

In [14]:
submission_5[sub.columns].to_csv('submission_fake_10.csv', index=False)

In [44]:
submission[sub.columns].to_csv('submission_12.csv', index=False)

In [47]:
check_score(submission_3)

AttributeError: ignored

In [43]:
submission

,phone,millisSinceGpsEpoch,latDeg,lngDeg,phoneName,collectionName
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.416386,-122.081262,Pixel4,2020-05-15-US-MTV-1
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.416470,-122.081532,Pixel4,2020-05-15-US-MTV-1
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.416538,-122.081750,Pixel4,2020-05-15-US-MTV-1
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.416588,-122.081910,Pixel4,2020-05-15-US-MTV-1
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.416622,-122.082018,Pixel4,2020-05-15-US-MTV-1
...,...,...,...,...,...,...
91481,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763185000,37.334547,-121.899402,SamsungS20Ultra,2021-04-29-US-SJC-3
91482,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763186000,37.334547,-121.899393,SamsungS20Ultra,2021-04-29-US-SJC-3
91483,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763187000,37.334548,-121.899383,SamsungS20Ultra,2021-04-29-US-SJC-3
91484,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763188000,37.334549,-121.899372,SamsungS20Ultra,2021-04-29-US-SJC-3


In [45]:
submission[sub.columns]

,phone,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.416386,-122.081262
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.416470,-122.081532
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.416538,-122.081750
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.416588,-122.081910
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.416622,-122.082018
...,...,...,...,...
91481,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763185000,37.334547,-121.899402
91482,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763186000,37.334547,-121.899393
91483,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763187000,37.334548,-121.899383
91484,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763188000,37.334549,-121.899372
